## Baixar o conjunto de dados

Ao executar as células a seguir, o conjunto de dados [harshsinghal/nlp-and-llm-related-arxiv-papers](https://www.kaggle.com/datasets/harshsinghal/nlp-and-llm-related-arxiv-papers), que contém artigos do [arXiv](https://arxiv.org/) relacionados a Processamento de Linguagem Natural (NLP) e Modelos de Linguagem de Grande Escala (LLM).

O conjunto ~ 970MB em arquivos PDF.

### Usando a API do Kaggle

Caso você prefira, pode usar a API do Kaggle. Será necessário seguir a [documentação](https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md#api-credentials): 

1. baixe o arquivo `kaggle.json` em https://www.kaggle.com/settings/account (na seção API).
2. Coloque-o em: 
   1. Windows: `C:\Users\<Windows-username>\.kaggle\kaggle.json`
   2. Linux: `$XDG_CONFIG_HOME/kaggle/kaggle.json` (em geral será `~/.config/kaggle/kaggle.json`).
   3. Outros: `~/.kaggle/kaggle.json`
3. Execute as células a seguir.

In [ ]:
%pip install --quiet kaggle

In [ ]:
from pathlib import Path

if not next(Path("./data/arxiv/files").iterdir(), None):
    import kaggle

    kaggle.api.authenticate()

    Path("./data/arxiv/files").mkdir(parents=True, exist_ok=True)
    kaggle.api.dataset_download_files('harshsinghal/nlp-and-llm-related-arxiv-papers', path='data/arxiv/files', unzip=True)

### Sem usar a API do Kaggle

Caso você prefira, pode baixar o conjunto de dados [aqui](https://www.kaggle.com/datasets/harshsinghal/nlp-and-llm-related-arxiv-papers). Descompacte-o para `/data/arxiv/files`.

## Analisando os dados

In [ ]:
%pip install langchain langchain_community langchain_chroma pypdf "unstructured[pdf]" pytesseract 

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Terminal ANSI colors
BOLD = "\033[1m"; RESET = "\033[0m"; GREEN = "\033[32m"; BLUE = "\033[34m"; RED = "\033[31m"

loader = PyPDFDirectoryLoader("./data/arxiv/files/")
print(f"Loading files...")
docs = loader.load()
print(f"{GREEN}Loaded {len(docs)} documents.{RESET}")

Loading files.


Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)


Loaded 388 documents.


In [ ]:
%pip install langchain_experimental langchain_openai

In [5]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings())
original_docs_len = len(docs)
docs = text_splitter.transform_documents(docs)
print(f"{GREEN}Separei {original_docs_len} documentos em {len(docs)} pedaços.{RESET}")

Separei 388 documentos em 1691 pedaços.


In [20]:
import chromadb
import tqdm

client = chromadb.PersistentClient(path="chromadb")
coll = client.get_or_create_collection("arxiv")

count = coll.count()
print(f"A coleção já contém {count} documentos." if count else "A coleção está vazia.")

ids = [f"id{i}" for i in range(len(docs))]

print(f"Começando a adicionar {len(docs)} documentos...")

batch_size = 30
# Load the documents in batches of batch_size
for i in tqdm.tqdm(
    range(0, len(docs), batch_size), desc="Adicionando documentos", unit_scale=batch_size
):

    coll.add(
        ids=ids[i : i+batch_size],
        documents=[d.page_content for d in docs[i : i + batch_size]],
    )

new_count = coll.count()
print(f"{GREEN}Adicionei {new_count - count} documentos.{RESET}")

A coleção está vazia.
Começando a adicionar 1691 documentos...


Adicionando documentos:   0%|          | 0/1710 [00:00<?, ?it/s]

Adicionando documentos: 100%|██████████| 1710/1710 [02:57<00:00,  9.62it/s]

Adicionei 1691 documentos


In [34]:
search = coll.query(
    query_texts=["qual a importância dos llms?"],
    n_results=5,
)

chars = sum(len(d) for d in search["documents"][0])
print(f"A pesquisa retornou {chars} caracteres.")

A pesquisa retornou 7702 caracteres.


In [46]:
from langchain_google_genai import ChatGoogleGenerativeAI

context = "\n".join(search["documents"][0])

llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro")

query = "qual a importância dos llms?"
prompt = f"Contexto: {context}\n\nPergunta: {query}\n\nResposta:"
print(f"Sending {len(prompt)} characters to the model...")

response = llm.invoke(prompt)
print(f"{BOLD}[User ] >>{RESET} {query}\n\n{BOLD}[Model] >>{RESET} {response.content}")

Sending 7767 characters to the model...
[User ] >> qual a importância dos llms?

[Model] >> O texto que você forneceu destaca a importância dos LLMs (Large Language Models) em diversos aspectos. Vamos resumir os pontos chave:

**1. Capacidade e Impacto:**

* **Compreensão e Geração de Linguagem:** LLMs demonstram uma capacidade impressionante de entender a linguagem natural e gerar textos coerentes e contextualmente relevantes.
* **Resolução de Tarefas Complexas:** Através da geração de texto, os LLMs podem ser utilizados para solucionar uma variedade de tarefas, desde tradução e resumo até programação e escrita criativa.
* **Avanços Significativos em IA:** O desenvolvimento de LLMs representa um avanço notável no campo da inteligência artificial, abrindo caminho para novas aplicações e pesquisas.

**2. Aplicações Práticas:**

* **Pesquisa Científica:** LLMs podem auxiliar em diversas etapas da pesquisa, como revisão de literatura, geração de hipóteses, análise de dados e escrita de ar